In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
from datetime import date
import uuid
import unidecode
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options

In [39]:

main_url = 'http://gaceta.diputados.gob.mx/base/inis/64/gp64_b_diputados.php3'

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(main_url)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87]


In [22]:
def obtain_iniciativas(driver, diputado, legis):
    all_iniciativas = []
    
    links, id_links = obtain_link_gaceta(driver)

    for i, x, y in zip(driver.find_elements_by_css_selector('p'), links, id_links):
        #temp_date = re.sub('Fecha: ', '', i.text.split('\n')[0])
        temp_date = re.sub('Fecha: ', '',driver.find_elements_by_css_selector('p')[0].text.split('\n')[0])
        
        temp = {}
        temp['legislatura'] = legis
        temp['name_inc'] = unidecode.unidecode(diputado)
        temp['year_iniciativa'] = datetime.date(int(temp_date.split('-')[0]),month_to_num(temp_date.split('-')[1].lower()), int(temp_date.split('-')[2])).year
        temp['month_iniciativa'] = datetime.date(int(temp_date.split('-')[0]),month_to_num(temp_date.split('-')[1].lower()), int(temp_date.split('-')[2])).month
        temp['day_iniciativa'] = datetime.date(int(temp_date.split('-')[0]),month_to_num(temp_date.split('-')[1].lower()), int(temp_date.split('-')[2])).day
        #temp['year_iniciativa'] = datetime.date(int(temp_date.split('/')[2]), int(temp_date.split('/')[0]), int(temp_date.split('/')[1])).year
        #temp['month_iniciativa'] = datetime.date(int(temp_date.split('/')[2]), int(temp_date.split('/')[0]), int(temp_date.split('/')[1])).month
        #temp['day_iniciativa'] =datetime.date(int(temp_date.split('/')[2]), int(temp_date.split('/')[0]), int(temp_date.split('/')[1])).day
        temp['topic_iniciativa'] = unidecode.unidecode(i.text.split('\n')[1])
        temp['link_gaceta'] = x
        temp['status_raw'] = i.text.split('\n')[-2:][0]
        temp['status_clean'] = obtain_status(i.text.split('\n')[-2:][0])
        temp['date_capture_status'] =  date.today().strftime('%Y-%m-%d')
        temp['id_iniciativa'] = str(y)
        all_iniciativas.append(temp)
    return(all_iniciativas)

In [21]:
re.sub('Fecha: ', '',driver.find_elements_by_css_selector('p')[0].text.split('\n')[0])

'2012-SEP-11'

In [4]:
def obtain_status(x):
    if any(re.findall('Turnada a la Cámara de Senadores', x)):
        return('dictaminada positivamente')
    elif any(re.findall('^Retirada', x)):
        return('retirada')
    elif any(re.findall('^Prórroga', x)):
        return('prorroga')
    elif any(re.findall('^Turnada a la Comisión', x)):
        return('turnada a comision')
    else:
        return(x)

In [5]:
def month_to_num(x):
    if x == 'ene':
        return(1)
    elif x == 'feb':
        return(2)
    elif x == 'mar':
        return(3)
    elif x == 'abr':
        return(4)
    elif x == 'may':
        return(5)
    elif x == 'jun':
        return(6)
    elif x == 'jul':
        return(7)
    elif x == 'ago':
        return(8)
    elif x == 'sep':
        return(9)
    elif x == 'oct':
        return(10)
    elif x == 'nov':
        return(11)
    elif x == 'dic':
        return(12)

In [7]:
def obtain_link_gaceta(driver):
    links = []
    id_inks = []
    for i in driver.find_elements_by_css_selector('p'):
        try:
            links.append(i.find_elements_by_css_selector('a')[-1:][0].get_attribute('href'))
            id_links.append(str(uuid.uuid3(uuid.NAMESPACE_URL,i.find_elements_by_css_selector('a')[-1:][0].get_attribute('href'))))
        except:
            links.append('')
            id_inks.append(str(uuid.uuid1()))
    return(links, id_inks)
            

In [36]:
links_legislaturas = [
    #'http://gaceta.diputados.gob.mx/base/inis/64/gp64_b_diputados.php3', 
#'http://gaceta.diputados.gob.mx/SIL/Iniciativas/63/gp63_b_diputados.php3', 
'http://gaceta.diputados.gob.mx/SIL/Iniciativas/62/gp62_b_diputados.php3',
'http://gaceta.diputados.gob.mx/SIL/Iniciativas/61/gp61_b_diputados.php3',
#'http://gaceta.diputados.gob.mx/SIL/Iniciativas/60/gp60_b_diputados.php3'
]

In [28]:
driver.get(links_legislaturas[2])

In [31]:
import json

In [33]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/03-legislation/' + name_file, 'w') as json_file:
    json.dump(data_legislatura, json_file)     

In [34]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/03-legislation/' + name_file) as json_file:
    data = json.load(json_file)     

In [40]:
data_all_legislaturas = []
for link, leg in tqdm(zip(links_legislaturas, [62, 61]), total = len(links_legislaturas)):
    driver.get(link)
    diputados = len(driver.find_elements_by_css_selector('option'))
    
    data_legislatura = []
    for i in tqdm(range(diputados)):
        driver.get(link)
        
        driver.find_elements_by_css_selector('option')[i].click()
        dip = driver.find_elements_by_css_selector('option')[i].text
        driver.find_element_by_css_selector('body > table:nth-child(3) > tbody > tr > td:nth-child(2) > input[type=SUBMIT]:nth-child(1)').click()
        
        try:
            iniciativas_dip = obtain_iniciativas(driver, dip, leg)
            for h in iniciativas_dip:
                data_legislatura.append(h)
        except:
            print(dip)
    
    data_all_legislaturas.append(data_legislatura)
    name_file = 'bills_proposed' + str(leg) + '.json'
    
    with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/03-legislation/' + name_file, 'w') as json_file:
        json.dump(data_legislatura, json_file)     

 33%|███▎      | 155/475 [03:37<07:47,  1.46s/it]


 65%|██████▌   | 309/475 [07:39<03:28,  1.25s/it]


 97%|█████████▋| 463/475 [10:53<00:16,  1.35s/it]


 27%|██▋       | 142/517 [03:27<10:14,  1.64s/it]


 57%|█████▋    | 296/517 [07:31<03:48,  1.03s/it]


 87%|████████▋ | 450/517 [11:17<02:08,  1.92s/it]


100%|██████████| 2/2 [23:48<00:00, 714.03s/it]


In [48]:
import json
name_file = 'bills_proposed' + str(leg) + '.json'
with open('../../data/03-legislation/' + name_file, 'w') as json_file:
    json.dump(iniciativas_dip, json_file)     